In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/train.csv")
# df.drop(columns=["id"], inplace=True)

In [3]:
df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 64 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year_Factor                75757 non-null  int64  
 1   State_Factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  float64
 6   energy_star_rating         49048 non-null  float64
 7   ELEVATION                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

In [5]:
df

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75752,6,State_11,Commercial,Office_Uncategorized,20410.0,1995.0,8.0,36.6,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,132.918411,75752
75753,6,State_11,Residential,5plus_Unit_Building,40489.0,1910.0,98.0,36.6,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,39.483672,75753
75754,6,State_11,Commercial,Commercial_Other,28072.0,1917.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,48.404398,75754
75755,6,State_11,Commercial,Commercial_Other,53575.0,2012.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,592.022750,75755


In [6]:
TARGET_COLUMN = "site_eui"

In [7]:
# numeric_data = df.select_dtypes(include=[np.number])
# categorical_data = df.select_dtypes(exclude=[np.number])

# numeric_features = numeric_data.columns.tolist()
# numeric_features.remove(TARGET_COLUMN)
# categorical_features = categorical_data.columns.tolist()

In [8]:
# numeric_features

In [9]:
# categorical_features

### Feature Engineering/Selection

In [10]:
df

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75752,6,State_11,Commercial,Office_Uncategorized,20410.0,1995.0,8.0,36.6,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,132.918411,75752
75753,6,State_11,Residential,5plus_Unit_Building,40489.0,1910.0,98.0,36.6,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,39.483672,75753
75754,6,State_11,Commercial,Commercial_Other,28072.0,1917.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,48.404398,75754
75755,6,State_11,Commercial,Commercial_Other,53575.0,2012.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,592.022750,75755


In [11]:
# Find Standard Deviation of min, max and avg temp among months
min_temps = [
    "january_min_temp",
    "february_min_temp",
    "march_min_temp",
    "april_min_temp",
    "may_min_temp",
    "june_min_temp",
    "july_min_temp",
    "august_min_temp",
    "september_min_temp",
    "october_min_temp",
    "november_min_temp",
    "december_min_temp"
]

max_temps = [
    "january_max_temp",
    "february_max_temp",
    "march_max_temp",
    "april_max_temp",
    "may_max_temp",
    "june_max_temp",
    "july_max_temp",
    "august_max_temp",
    "september_max_temp",
    "october_max_temp",
    "november_max_temp",
    "december_max_temp"
]

avg_temps = [
    "january_avg_temp",
    "february_avg_temp",
    "march_avg_temp",
    "april_avg_temp",
    "may_avg_temp",
    "june_avg_temp",
    "july_avg_temp",
    "august_avg_temp",
    "september_avg_temp",
    "october_avg_temp",
    "november_avg_temp",
    "december_avg_temp",
]

df["min_temp_std"] = df[min_temps].T.std()
df["max_temp_std"] = df[max_temps].T.std()
df["avg_temp_std"] = df[avg_temps].T.std()

In [12]:
df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id,min_temp_std,max_temp_std,avg_temp_std
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,0,1.0,1.0,1.0,NaN,248.682615,0,6.575897,7.432525,5.500612
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,0,1.0,NaN,1.0,12.0,26.500150,1,6.575897,7.432525,5.500612
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,0,1.0,NaN,1.0,12.0,24.693619,2,6.575897,7.432525,5.500612
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,0,1.0,NaN,1.0,12.0,48.406926,3,6.575897,7.432525,5.500612
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,0,1.0,1.0,1.0,NaN,3.899395,4,6.575897,7.432525,5.500612


In [13]:
# Find monthwise difference between the temperatures

In [14]:
# days_above ranges
# days_below ranges
# find their sums to see if they are yearly/monthly/etc.
days_above_below = [
    "days_below_30F",
    "days_below_20F",
    "days_below_10F",
    "days_below_0F",
    "days_above_80F",
    "days_above_90F",
    "days_above_100F",
    "days_above_110F",
]

df["0-10"] = df["days_below_10F"] - df["days_below_0F"]
df["10-20"] = df["days_below_20F"] - df["days_below_10F"]
df["20-30"] = df["days_below_30F"] - df["days_below_20F"]
df["80-90"] = df["days_above_80F"] - df["days_above_90F"]
df["90-100"] = df["days_above_90F"] - df["days_above_100F"]
df["100-110"] = df["days_above_100F"] - df["days_above_110F"]
df["30-80"] = (366 - df[days_above_below].sum(axis=1)).clip(lower=0)

In [15]:
# Do something about missing values in direction_max_wind_speed and direction_peak_wind_speed

In [16]:
numeric_features = [
    "floor_area",
    "year_built",
    "energy_star_rating",
    "ELEVATION",
    
    # "january_min_temp",
    # "january_avg_temp",
    # "january_max_temp",
    # "february_min_temp",
    # "february_avg_temp",
    # "february_max_temp",
    # "march_min_temp",
    # "march_avg_temp",
    # "march_max_temp",
    # "april_min_temp",
    # "april_avg_temp",
    # "april_max_temp",
    # "may_min_temp",
    # "may_avg_temp",
    # "may_max_temp",
    # "june_min_temp",
    # "june_avg_temp",
    # "june_max_temp",
    # "july_min_temp",
    # "july_avg_temp",
    # "july_max_temp",
    # "august_min_temp",
    # "august_avg_temp",
    # "august_max_temp",
    # "september_min_temp",
    # "september_avg_temp",
    # "september_max_temp",
    # "october_min_temp",
    # "october_avg_temp",
    # "october_max_temp",
    # "november_min_temp",
    # "november_avg_temp",
    # "november_max_temp",
    # "december_min_temp",
    # "december_avg_temp",
    # "december_max_temp",
    
    "cooling_degree_days",
    "heating_degree_days",
    "precipitation_inches",
    "snowfall_inches",
    "snowdepth_inches",
    "avg_temp",
    "days_below_30F",
    "days_below_20F",
    "days_below_10F",
    "days_below_0F",
    "days_above_80F",
    "days_above_90F",
    "days_above_100F",
    "days_above_110F",
    "max_wind_speed",
    "days_with_fog",
    "min_temp_std",
    "max_temp_std",
    "avg_temp_std",
    "0-10",
    "10-20",
    "20-30",
    "30-80",
    "80-90",
    "90-100",
    "100-110"
] + min_temps + max_temps + avg_temps

categorical_features = [
    "Year_Factor",
    "State_Factor",
    "building_class",
    "facility_type",
    "direction_max_wind_speed",
    "direction_peak_wind_speed"
]

drop_columns = [
    "id"
]

In [17]:
len(numeric_features) + len(categorical_features) + len(drop_columns)

73

In [18]:
# ind = 58
# print(numeric_features[ind])
# print(X_train[numeric_features[ind]].value_counts().size)
# print()
# print(X_train[numeric_features[ind]].value_counts())

## Model

In [19]:
from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    train_test_split,
    RandomizedSearchCV
)

from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer,
    PolynomialFeatures
)

from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import StackingClassifier

In [20]:
import time

In [21]:
train_df, val_df = train_test_split(
    df,
    test_size=0.1,
    random_state=123
)

X_train, y_train = train_df.drop(columns=[TARGET_COLUMN]), train_df[TARGET_COLUMN]
X_val, y_val = val_df.drop(columns=[TARGET_COLUMN]), val_df[TARGET_COLUMN]

In [22]:
for dt in [X_train, X_val, y_train, y_val]:
    print(dt.shape)

(68181, 73)
(7576, 73)
(68181,)
(7576,)


In [23]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score=False):

    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_val": model.score(X_val, y_val),
        "mse_val": mean_squared_error(y_val, y_val_pred),
        "mape_val": mean_absolute_percentage_error(y_val, y_val_pred)
    }

    if return_train_score:
        y_train_pred = model.predict(X_train)

        score_dict["r2_train"] = model.score(X_train, y_train)
        score_dict["mse_train"] = mean_squared_error(y_train, y_train_pred)
        score_dict["mape_train"] = mean_absolute_percentage_error(y_train, y_train_pred)

    scores_result = pd.Series(score_dict)

    return model, scores_result

In [24]:
from fancyimpute import IterativeImputer
mice_imp = IterativeImputer()
# X_train_mice = X_train_numeric.copy()
# X_val_mice = X_val_numeric.copy()
# X_train_mice.iloc[:, :] = mice_imp.fit_transform(X_train_mice)
# X_val_mice.iloc[:, :] = mice_imp.transform(X_val_mice)

In [25]:
from fancyimpute import KNN

knn = KNN(2)
# X_train_categorical = X_train_categorical.copy()
# X_val_categorical = X_val_categorical.copy()
# X_train_categorical.iloc[:, :] = knn.fit_transform(X_train_categorical)
# X_val_categorical.iloc[:, :] = knn.transform(X_val_categorical)

In [26]:
pipe_numeric_feats = make_pipeline(
    mice_imp,
    StandardScaler()
)

pipe_cat_feats = make_pipeline(
    knn,
    OneHotEncoder()
)

column_transformer = make_column_transformer(
    (pipe_numeric_feats, numeric_features),
    (OneHotEncoder(), categorical_features),
    ("drop", drop_columns)
)

In [27]:
X_transformed = column_transformer.fit_transform(X_train)
X_transformed

array([[-0.43079795, -0.61896931,  0.1141558 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.4492352 ,  0.93954345,  0.25344281, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.03223259,  0.56335072, -0.02627946, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.8154593 ,  0.34838344,  0.18202486, ...,  0.        ,
         0.        ,  1.        ],
       [-0.41593468, -1.04890386,  1.05448594, ...,  0.        ,
         0.        ,  1.        ],
       [-0.19757276, -1.04890386, -0.1096181 , ...,  0.        ,
         0.        ,  0.        ]])

In [28]:
pipe_lr = make_pipeline(column_transformer, Ridge(max_iter=10000))
pipe_dt = make_pipeline(column_transformer, DecisionTreeRegressor())
# pipe_svc = make_pipeline(column_transformer, SVC())
pipe_rf = make_pipeline(column_transformer, RandomForestRegressor())
pipe_xgb = make_pipeline(column_transformer, XGBRegressor(verbosity=0))
pipe_lgbm = make_pipeline(column_transformer, LGBMRegressor())
pipe_catboost = make_pipeline(column_transformer, CatBoostRegressor(verbose=False))

In [29]:
models = {
    "Ridge": pipe_lr,
    "Decision Tree": pipe_dt,
    "Random Forest": pipe_rf,
    "XGB": pipe_xgb,
    "LGBM": pipe_lgbm,
    "Cat Boost": pipe_catboost
}

In [34]:
results = {}

for name, model in models.items():
    print(f"Start {name}!")
    _, results[name] = cross_val_scores(
        model,
        X_train,
        y_train,
        X_val,
        y_val,
        return_train_score=True
    )

    print(f"Done {name}!")


Start Ridge!
Done Ridge!
Start Decision Tree!
Done Decision Tree!
Start Random Forest!
Done Random Forest!
Start XGB!
Done XGB!
Start LGBM!
Done LGBM!
Start Cat Boost!
Done Cat Boost!


In [35]:
pd.DataFrame(results)

,Ridge,Decision Tree,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.386984,0.055450,0.508355,0.525897,0.490960,0.519719
mse_val,1903.313590,2932.672655,1526.476743,1472.011770,1580.484354,1491.191617
mape_val,0.512406,0.476046,0.407606,0.422695,0.449780,0.429577
r2_train,0.357068,0.999416,0.930254,0.683156,0.544691,0.635207
mse_train,2202.494467,2.001399,238.929027,1085.413835,1559.754283,1249.671528
mape_train,0.590746,0.001816,0.171319,0.435871,0.494596,0.455505


## FCNN


In [36]:
import numpy as np
import pandas as pd
from torchvision import transforms, datasets, utils
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
import torch.utils.data as data
from torch import nn

In [37]:
train_path = "data/train.csv"
test_path = "data/test.csv"

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 74 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year_Factor                75757 non-null  int64  
 1   State_Factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  float64
 6   energy_star_rating         49048 non-null  float64
 7   ELEVATION                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

In [40]:
class Dataprep(Dataset):
    
    def __init__(self, train_path, test_path, size=0.1, is_train=True):
        df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        
        TARGET_COLUMN = "site_eui"

        self.is_train = is_train

        train_df, val_df = train_test_split(
            df,
            test_size=0.1,
            random_state=123
        )

        self.X_train, self.y_train = train_df.drop(columns=[TARGET_COLUMN]), train_df[TARGET_COLUMN]
        self.X_val, self.y_val = val_df.drop(columns=[TARGET_COLUMN]), val_df[TARGET_COLUMN]
        self.X_test = test_df


    def engineer_features(self):
        # Find Standard Deviation of min, max and avg temp among months
        min_temps = [
            "january_min_temp",
            "february_min_temp",
            "march_min_temp",
            "april_min_temp",
            "may_min_temp",
            "june_min_temp",
            "july_min_temp",
            "august_min_temp",
            "september_min_temp",
            "october_min_temp",
            "november_min_temp",
            "december_min_temp"
        ]

        max_temps = [
            "january_max_temp",
            "february_max_temp",
            "march_max_temp",
            "april_max_temp",
            "may_max_temp",
            "june_max_temp",
            "july_max_temp",
            "august_max_temp",
            "september_max_temp",
            "october_max_temp",
            "november_max_temp",
            "december_max_temp"
        ]

        avg_temps = [
            "january_avg_temp",
            "february_avg_temp",
            "march_avg_temp",
            "april_avg_temp",
            "may_avg_temp",
            "june_avg_temp",
            "july_avg_temp",
            "august_avg_temp",
            "september_avg_temp",
            "october_avg_temp",
            "november_avg_temp",
            "december_avg_temp",
        ]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["min_temp_std"] = X[min_temps].T.std()
            X["max_temp_std"] = X[max_temps].T.std()
            X["avg_temp_std"] = X[avg_temps].T.std()

        days_above_below = [
            "days_below_30F",
            "days_below_20F",
            "days_below_10F",
            "days_below_0F",
            "days_above_80F",
            "days_above_90F",
            "days_above_100F",
            "days_above_110F",
        ]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["0-10"] = X["days_below_10F"] - X["days_below_0F"]
            X["10-20"] = X["days_below_20F"] - X["days_below_10F"]
            X["20-30"] = X["days_below_30F"] - X["days_below_20F"]
            X["80-90"] = X["days_above_80F"] - X["days_above_90F"]
            X["90-100"] = X["days_above_90F"] - X["days_above_100F"]
            X["100-110"] = X["days_above_100F"] - X["days_above_110F"]
            X["30-80"] = (366 - X[days_above_below].sum(axis=1)).clip(lower=0)


    def preprocess(self):
        pipe_numeric_feats = make_pipeline(
           SimpleImputer(strategy="mean"),
           StandardScaler()
        )
        column_transformer = make_column_transformer(
            (pipe_numeric_feats, numeric_features),
            (OneHotEncoder(handle_unknown = "ignore"), categorical_features)
        )
        
        self.X_train = column_transformer.fit_transform(self.X_train)
        self.X_val = column_transformer.transform(self.X_val)
        self.X_test = column_transformer.transform(self.X_test)

        self.X_train = torch.tensor(self.X_train)
        self.y_train = torch.tensor(self.y_train.values)
        self.X_val = torch.tensor(self.X_val)
        self.y_val = torch.tensor(self.y_val.values)
        self.X_test = torch.tensor(self.X_test)        

feature_set = Dataprep(train_path, test_path)
feature_set.engineer_features()
feature_set.preprocess()

In [41]:
def linear_block(input_size, output_size):
    return nn.Sequential(
        nn.Linear(input_size, output_size),
        nn.ReLU(),
        nn.Dropout(0.2)
    )

class EnergyRegressor(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            linear_block(input_size, 2 * input_size),
            linear_block(2 * input_size, 3 * input_size),
            linear_block(3 * input_size, 1 * input_size),
            nn.Linear(input_size, 256),
            nn.Linear(256, 128),
            nn.Linear(128, 64),
            nn.Linear(64, 1)
        )
    
    def forward(self, X):
        X = self.layers(X)
        return X
        

In [42]:
trainloader = DataLoader(TensorDataset(feature_set.X_train, feature_set.y_train), batch_size=1, shuffle=True)
validloader = DataLoader(TensorDataset(feature_set.X_val, feature_set.y_val), batch_size=1, shuffle=True)

In [43]:
model = EnergyRegressor(feature_set.X_train.shape[1])
model.to(device)

EnergyRegressor(
  (layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=161, out_features=322, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=322, out_features=483, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=483, out_features=161, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (3): Linear(in_features=161, out_features=256, bias=True)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): Linear(in_features=128, out_features=64, bias=True)
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [44]:
def r2score(predictions, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

def mse(predictions, targets):
    return torch.mean((predictions - targets) ** 2)
        
def rmse(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())


In [47]:
def trainer(model, criterion, optimizer, trainloader, validloader, epochs):
    
    for epoch in range(epochs):
        train_batch_mse = []
        train_batch_rsme = []
        train_batch_r2 = []
        val_batch_mse = []
        val_batch_rsme = []
        val_batch_r2 = []
        # labels = []
        # preds = []
        # labels_val = []
        # preds_val = []
        
        for X, y in trainloader:
            optimizer.zero_grad()
            X = X.to(device)
            y = y.to(device)
            y_hat = model(X.type(torch.float32)) 
            loss = criterion(y_hat, y.type(torch.float32))
            loss = loss.to(device)
            loss.backward()
            optimizer.step()
            mse_train = mse(y_hat, y.type(torch.float32))
            rsme_train = rmse(y_hat, y.type(torch.float32))
            r2_train = r2score(y_hat, y.type(torch.float32))
            train_batch_mse.append(mse_train)
            train_batch_rsme.append(rsme_train)
            train_batch_r2.append(r2_train)
            # labels.extend(y.tolist())
            # preds.extend(y_hat.tolist())

#         train_mse = mse(torch.FloatTensor(labels), torch.FloatTensor(preds))
#         train_rmse = rmse(torch.FloatTensor(labels), torch.FloatTensor(preds))
#         train_r2 = r2score(torch.FloatTensor(labels), torch.FloatTensor(preds))

        train_mse = torch.sum(torch.Tensor(train_batch_mse)) / len(trainloader)
        train_rsme = torch.sum(torch.Tensor(train_batch_rsme)) / len(trainloader)
        train_r2 = torch.sum(torch.Tensor(train_batch_r2)) / len(trainloader)
        model.eval()
        with torch.no_grad():  
            for X_valid, y_valid in validloader:
                X_valid = X_valid.to(device)
                y_valid = y_valid.to(device)
                y_hat_val = model(X_valid.type(torch.float32)) 
                mse_val = mse(y_hat_val, y_valid.type(torch.float32))
                rsme_val = rmse(y_hat_val, y_valid.type(torch.float32))
                r2_val = r2score(y_hat_val, y_valid.type(torch.float32))
                val_batch_mse.append(mse_val)
                val_batch_rsme.append(rsme_val)
                val_batch_r2.append(r2_val)
                # labels_val.extend(y_valid.tolist())
                # preds_val.extend(y_hat_val.tolist())

            # val_mse = mse(torch.FloatTensor(labels_val), torch.FloatTensor(preds_val))
            # val_rsme = rmse(torch.FloatTensor(labels_val), torch.FloatTensor(preds_val))
            # val_r2 = r2score(torch.FloatTensor(labels_val), torch.FloatTensor(preds_val))
            val_mse = torch.sum(torch.Tensor(val_batch_mse)) / len(validloader)
            val_rsme = torch.sum(torch.Tensor(val_batch_rsme)) / len(validloader)
            val_r2 = torch.sum(torch.Tensor(val_batch_r2)) / len(validloader) 
        model.train()
        
        print(f"Epoch {epoch + 1}: ")
        print(f"Train MSE: {train_mse}. Train RMSE: {train_mse}, Train R2: {train_r2}.")
        print(f"Val MSE: {val_mse}, Val RMSE: {val_rsme}, Val R2: {val_r2}.")

In [48]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
trainer(model, criterion, optimizer, trainloader, validloader, epochs=5)

Epoch 1: 
Train MSE: 110045685612544.0. Train RMSE: 110045685612544.0, Train R2: -inf.
Val MSE: 32700824.0, Val RMSE: 5718.19287109375, Val R2: -inf.
Epoch 2: 
Train MSE: 1483470976.0. Train RMSE: 1483470976.0, Train R2: -inf.
Val MSE: 4518.4541015625, Val RMSE: 52.99092483520508, Val R2: -inf.
Epoch 3: 
Train MSE: 79345000.0. Train RMSE: 79345000.0, Train R2: -inf.
Val MSE: 4050.53369140625, Val RMSE: 39.6556396484375, Val R2: -inf.


KeyboardInterrupt: 

In [ ]:
def predict(X):
    return model(X.type(torch.float32))  

In [ ]:
predict(feature_set.X_test)